![The glorious SpatialIndex logo](img/spatial_index_logo.png)
# SpatialIndex Basic Tutorial with Jupyter Notebook

## Introduction

SpatialIndex is the best choice for indexing and querying large circuits. It provides support for soma, segment and synapse circuits.

Are you confused about all of this? Let's start from the beginning. 

Let's say you need to look into your circuit and find all the elements that are present in a certain region of space. This region, in SpatialIndex, could be a 3D box or a sphere.
First of all we need to "index" the circuit. The indexing is a process in which we reorganize the content of the circuit so that it's much easier and quicker to look into. The indexing needs to be performed in order for SpatialIndex to run "queries" i.e. interrogate the circuit for the presence of elements in a certain region of space. The good news is that we need to run the indexing **only once**, then we can save it to disk so it can be loaded much faster in future iterations.

Once the circuit has been indexed, we can perform queries on it by specifying the coordinates of the vertices of the 3D box where we want to find elements. You can also perform "proximity queries" i.e. find all the elements that are close to a certain point. In that case we need to specify the coordinates of the center and the radius of the sphere around it in which we want to look.

In this tutorial we'll go through the entire process of setting up SpatialIndex, index a circuit and then perform some queries. What will be shown here is just the typical SpatialIndex workflow but keep in mind that there's much more you can do with it. For that we point you to the [official SpatialIndex documentation](https://bbpteam.epfl.ch/documentation/projects/spatial-index/latest/intro.html) and the examples in the `example` folder of the SpatialIndex repo.

Let's start!

## Installation

For the purposes of this tutorial, we will be using the `module load` command to load and setup SpatialIndex. This is the recommended way to use SpatialIndex on BB5 but you can also install it using `pip`, using `spack` or just by downloading and compiling the code yourself and creating your Python virtual environment. 

In [ ]:
%%bash
module load unstable spatial-index

Now that SpatialIndex has been loaded, we can proceed and generate the index for a small test circuit. Indexing can be performed in two different ways: from command line or using Python code. Let's start with the first approach.

## Your first query

Now that SpatialIndex has been loaded, we can proceed and generate the index for a small test circuit. Indexing can be performed in two different ways: from command line or using Python code. Let's start with the first approach.

The easiest way to do this is by using the `spatial-index-circuit` command line tool. While using this tool you just need to specify what kind of circuit you want to index (segment or synapse) and then the path to a SONATA `.json` file. The tool will automatically fetch the information regarding the location of the circuit and morphology files from the `.json` file.

You can get more information on this command by running:

In [ ]:
%%bash
spatial-index-circuit --help

We can now try to index our first circuit using this command like shown below. We use the `-o` flag to specify the name of the output file and let the command to the rest:

In [ ]:
%%bash
spatial-index-circuit segments /gpfs/bbp.cscs.ch/project/proj12/spatial_index/tutorial/usecases/usecase1/circuit_sonata.json -o usecase1.spi

When the process it's over, it should result in a file called `circuit2k.spi` that contains the index for the whole tree.

**You've just generated your first index!**

Before doing some queries, let's look at an alternative way of generating the index whenever you don't have access to a `.json` file containing all the circuit configuration.

---

In case we don't have a `.json` file relative to the circuit, in order to index a segment circuit we need two paths: the first path should point to the `circuit.mvd3` file of the circuit and the second path should point to the morphology folder. This information should be known to you and depends, of course, on the circuit you're studying. Otherwise you can *typically* find it in the `BlueConfig` or `CircuitConfig` file relative to the circuit you want to analyze.

For the purposes of this tutorial we will be using the `scx-2k-v6` circuit found in the `blueconfigs` repository. If one opens the `BlueConfig` file will find in the first line:

```
  1 Run Default
  2 {
  3     CircuitPath /gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k
  4     nrnPath /gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k/touches/functional
  5     MorphologyPath /gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k/morphologies
```

Both paths are spelled right in the first lines. Therefore our morphology path will be:

`/gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k/morphologies/ascii`

(we add `ascii` at the end of the path since we are interested in that format for morphologies)

and our nodes path will be:

`/gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k/circuit.mvd3`.

While the morphology path can also be a folder, the nodes path has to be an mvd3 file, typically called `circuit.mvd3`. Please check your case for the existence of an mvd3 nodes file in the folder and add its name to the path.

Also for this command we can use the `-o` option to specify the name and the path of the output index file.

We can now execute a terminal command to create the index. This is the required syntax:

```
Usage:
        spatial-index-nodes [options] <nodes-file> <morphology-dir>
        spatial-index-nodes --help
```

In the case of this example we run (please note that this can take a couple of minutes):

In [ ]:
%%bash
spatial-index-nodes /gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k/circuit.mvd3 /gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k/morphologies/ascii -o circuit2k.spi

When the process it's over, it should result in a file called `circuit2k.spi` that contains the index for the whole tree.

Now that we have an index we can query it for some information.

The simplest form of query would be a "proximity query" that identifies all the elements that are close by a certain radius from a certain point.

In order to do so we can first load the index itself and then query it using the `find_intersecting` function.
Let's load some libraries, our previously generated index and the `find_intersecting` function to perform our first query.

Let's start by loading the `MorphIndexBuilder` object from the `spatial_index` library, and then using the `load_dump` function to load the previously generated index

In [ ]:
from spatial_index import MorphIndexBuilder

In [ ]:
indexer = MorphIndexBuilder.load_dump("circuit2k.spi")

This should take just a dozen of seconds. Then we can proceed with the query itself.

For this test we want to look for all the elements that are closer than 50 units from the center of the axes.

That's pretty simple to achieve. Let's run this code:

In [ ]:
center = [0, 0, 0]
radius = 50
indexer.find_intersecting(center, radius)

We specified a `center` (an array containing the xyz coordinates of the "starting" point) and a `radius` that defines the range of the search.

The results are outputted as a numpy array of arrays containing the `gid`, `section_id` and `segment_id` of the results.

**CONGRATULATIONS! You just performed your first query!** 🥳🥳🥳

---

Now let's proceed with something slightly different.

The most common type of queries would be a box query i.e. asking for elements in a certain cubic area of space. In order to do so you have multiple choice but for the moment we'll show you the easiest way to get the data.

You can use the `find_intersecting_window_np` function to output the results of the box query as a dictionary of numpy arrays that can be easily manipulated and imported in a data analysis framework. 

We will define the two vertices of the box we want to query and then call the function.
Let's run this:

In [ ]:
min_corner = [0, 0, 0]
max_corner = [30, 30, 30]
indexer.find_intersecting_window_np(min_corner, max_corner)

As you can see it returns all the elements in that box with their relative data.

In the case of a segment (like in this case), the fields would be `gid`, `section_id`, `segment_id`, `radius`, `endpoint1`, `endpoint2` and `kind` which is `0` for a Soma, `1` for a Segment and `2` for a Synapse.

**Now you know the basics of SpatialIndex!**

---

For the next tutorial we will show some of the other tools and workflow you can use.

## Alternative way of creating an index

As shown before, you can create an index directly from the command line. This is useful especially in situations where you need to create a big index once and then use it multiple times for your queries. But there are situation where it can also be useful to create the index directly on the fly and also having the possibility to restrict the range of the index for specific use cases or testing.

In that case you can create the index from a Python script and query it on the fly. The index can be left in memory for the scope of the script or it can be dumped to file for future use. Let's see how this can be achieved.

We will be using the same circuit file as the example above so let's start by creating two variables containing the circuit and morphology paths:

In [ ]:
CIRCUIT_2K = "/gpfs/bbp.cscs.ch/project/proj12/jenkins/cellular/circuit-2k"
NODE_FILE = CIRCUIT_2K + "/circuit.mvd3"
MORPH_FILE = CIRCUIT_2K + "/morphologies/ascii"

Now we can create an index builder object and then process the relevant entries of the circuit. We can use the `process_all()` method to process ALL the entries (but this can take a lot of memory and a long time to run) or specify a range of entries to process if we're only interested in those. In this case you can use the `process_range` method.

For the sake of simplicity, in this example we will be processing just 50 cells using the `process_range` method. 

In [ ]:
indexer = MorphIndexBuilder(MORPH_FILE, NODE_FILE)
indexer.process_range((700, 750))  # 50 cells
# indexer.process_all()  # Processes all cells. Warning: Might exhaust memory and take a long time.

Now we can check that the index was built correctly by printing the number of elements in the index.

In [ ]:
print("Index contains", len(indexer.index), "elements. Saving to disk")

As you can see 50 cells generated more than 500k elements as each cells contains a soma but multiple segments.

After building the index we can now query it normally like the index we previously created via command line and then imported in SpatialIndex.

In [ ]:
min_corner = [0, 0, 0]
max_corner = [30, 30, 30]
indexer.index.find_intersecting_window_np(min_corner, max_corner)

Easy right?

Now, if you want, we can dump the index generated inside the script to disk for further use. This is not necessary and certain use cases or test can keep the index just in memory.

If you want to dump the index to file you can simply write:

In [ ]:
indexer.index.dump("example_segment_index.spi")

## Alternative queries

In the previous paragraph we already showed some ways of performing queries namely by using the `find_intersecting` function for vicinity queries and the `find_intersecting_window_np` function for box queries. However, there are multiple ways you can get the data you want, especially in situations where you don't need all the data related to a specific entry.

For example if you just need the ids of the entries (`gid`, `section_id`, `segment_id` for segments or `id`, `post_gid`, `pre_gid` for synapses) you can use the faster and sleeker function `find_intersecting_window` to get just that as a set of 3 numpy arrays. 

In [ ]:
ids = indexer.index.find_intersecting_window(min_corner, max_corner)
print("Number of elements within window:", len(ids))
print(ids)

Similarly if you just need the positions of the center of the elements in your query, you can use `find_intersecting_window_pos` like shown below:

In [ ]:
pos = indexer.index.find_intersecting_window_pos(min_corner, max_corner)
print(pos)

As you can see it returns a set of numpy arrays that contain the 3D position of the center of the elements.

If you prefer to output a sleeker, smaller entity for performance purposes, you can output the results of the query as slim objects. Then you can iterate over these objects to get the data you need in a second moment. In order to do so, you can use the `find_intersecting_window_objs` function like this:

In [ ]:
found_objects = indexer.index.find_intersecting_window_objs(min_corner, max_corner)

for i, obj in enumerate(found_objects):
    object_ids = obj.ids  # as tuple of gid, section, segment  # noqa
    # Individual propertioes
    print("Segment ids:", obj.gid, obj.section_id, obj.segment_id,
            "Centroid:", obj.centroid)
    if i >= 20:
        print("...")
        break

You can also just count the number of elements that are present in a certain region of space. If you're not interested in the actual details of the elements, you can just use the `count_intersecting()` function like shown below:

In [ ]:
count = indexer.index.count_intersecting(min_corner, max_corner)
print(count)

**With this you know most of the stuff necessary for you day-to-day use of SpatialIndex.**
**Well done! 🎉🎉🎉**

Thanks for following up until this point!

If you want you can now move to the `advanced_tutorial.ipynb` you can find in the `example` folder, where we'll go into more details on techniques like Multi-Indexing and Memory Mapping.

## Appendix I: Example list and description

The step-by-step tutorial is a very nice starting point but we included a lot of examples ready to use in the `examples` folder of the `SpatialIndex` repo.

If you're struggling with any of the snippets of code in this tutorial, or you simply want a more encompassing picture of the tool, please consider looking also at the examples. Here'we'll provide a list of the example files included and what they implement, hoping it can be useful.

- `basic_tutorial.ipynb`: this file!
- `advanced_tutorial.ipynb`: a Jupyter Notebook with examples on multi-indexing and memory mapping
- `segment_index.py`: simple indexing and querying of a segment index 
- `synapses_index.py`: simple indexing and querying of a synapse index
- `segment_index_sonata.py`: indexing and querying of a segment index using SONATA files
- `segment_multi_index_sonata.py`: indexing and querying of a segment multi-index using SONATA files
- `synapse_multi_index_sonata.py`: indexing and querying of a synapse multi-index using SONATA files
- `memory_map_index_seg.sh`: shell script to be run with `sbatch` that shows how to properly start Memory-Mapped indexing for a segment index
- `memory_map_index_seg.py`: Python script illustrating how to create a memory mapped index for segments, loading it from file and querying it
- `memory_map_index_syn.sh`: shell script to be run with `sbatch` that shows how to properly start Memory-Mapped indexing for a synapse
- `memory_map_index_syn.py`: Python script illustrating how to create a memory mapped index for synapses, loading it from file and querying it